# users table에 패스워드 넣기

In [50]:
import json
import pymysql
import pandas as pd

with open('data/mysql.json', 'r') as file: 
    config_str = file.read() 
    config = json.loads(config_str)

conn = pymysql.connect(
    host = config['host'],
    user = config['user'],
    password=config['password'],
    database=config['database'],
    port=config['port']
) 

## 패스워드 암호화
- secure hash algorithm(SHA-256) 단방향 평문 --> 비문 만드는 알고리즘

In [51]:
import hashlib
pwd = '1234'
pwd_sha256 = hashlib.sha256(pwd.encode())
pwd_sha256.digest()

b'\x03\xacgB\x16\xf3\xe1\\v\x1e\xe1\xa5\xe2U\xf0g\x956#\xc8\xb3\x88\xb4E\x9e\x13\xf9x\xd7\xc8F\xf4'

## Base64 로 인코딩

In [52]:
import base64
base64.b64encode(pwd_sha256.digest()) #바이트형식

b'A6xnQhbz4Vx2HuGl4lXwZ5U2I8iziLRFnhP5eNfIRvQ='

In [53]:
hash_pwd = base64.b64encode(pwd_sha256.digest()).decode('utf-8')
hash_pwd #스트링 형식

'A6xnQhbz4Vx2HuGl4lXwZ5U2I8iziLRFnhP5eNfIRvQ='

In [54]:
len(hash_pwd) #이래서 비밀번호 길이를 44로 해놓는다.

44

## 패스워드를 생성하는 함수

In [55]:
import hashlib
import base64
def gen_pwd(pwd):
    pwd_sha256 = hashlib.sha256(pwd.encode())
    hash_pwd = base64.b64encode(pwd_sha256.digest()).decode('utf-8')
    return hash_pwd

In [56]:
gen_pwd('rhyou10')

'xmisexq+qMxqKkvf6dk8n0nPWJ1zGhM8UsrJmzXaXTc='

## user table의 패스워드 넣기
- uid + 21 ==> pwd 설정한다는 가정

In [57]:
sql = """
    select uid
    from users
"""
cur = conn.cursor()
cur.execute(sql)
result = cur.fetchall()
result

(('admin',),
 ('djy',),
 ('eskim',),
 ('gdhong',),
 ('gdhong2',),
 ('jbpark',),
 ('jbpark2',),
 ('wjlee',))

In [58]:
sql = """
    UPDATE users 
    SET pwd=%s
    WHERE uid=%s;
"""
for line in result:
    uid = line[0]
    pwd = gen_pwd(uid+'21')
    cur.execute(sql,(pwd, uid))

conn.commit()  

In [59]:
sql_email = 'update users set email=%s where uid=%s;'
for line in result:
    uid = line[0]
    email = uid + '@mc.com'
    cur.execute(sql_email, (email, uid))
conn.commit()  

## 로그인

### case 1: uid가 없는 경우

In [60]:
sql_login = """
    select uid, pwd
    from users
    where uid = %s and is_deleted= 0
"""

In [61]:
#uid 임시설정
#없는경우 result는 null이 된다
uid = 'fool'
cur.execute(sql_login,(uid,))
result = cur.fetchone()
result

### case 2: 비밀번호가 틀린 경우

In [62]:
uid, pwd = 'djy', 'djy20'
cur.execute(sql_login,(uid,))
result = cur.fetchone()
result

('djy', 'mxsnWYLpB/nAOt37fpt3PMXpGGQprktT3vm2p9RfFyY=')

In [63]:
db_pwd = result[1]
hash_pwd = gen_pwd(pwd)
if db_pwd == hash_pwd:
    print('login')
else:
    print('password 틀림')

password 틀림


### case 3: 둘다 맞은경우

In [64]:
uid, pwd = 'djy', 'djy21'
cur.execute(sql_login,(uid,))
result = cur.fetchone()
result

('djy', 'mxsnWYLpB/nAOt37fpt3PMXpGGQprktT3vm2p9RfFyY=')

In [65]:
db_pwd = result[1]
hash_pwd = gen_pwd(pwd)
if db_pwd == hash_pwd:
    print('login')
else:
    print('password 틀림')

login


### 3가지 합침

In [66]:
uid, pwd = 'djy', 'djy21'
cur.execute(sql_login,(uid,))
result = cur.fetchone()
if result: #없으면 아이디가 없어서 null값 리턴
    db_pwd = result[1]
    hash_pwd = gen_pwd(pwd)
    if db_pwd == hash_pwd:
        print('login')
    else:
        print('password 틀림')
else:
    print('잘못된 아이디')

login


### 예외처리를 통해 해결한는 경우

In [67]:
uid, pwd = 'djy', 'djy21'
cur.execute(sql_login,(uid,))
result = cur.fetchone()
try: #없으면 아이디가 없어서 null값 리턴
    db_pwd = result[1] #result가 비어잆어서 try 가능
    hash_pwd = gen_pwd(pwd)
    if db_pwd == hash_pwd:
        print('login')
    else:
        print('password 틀림')
except:
    print('잘못된 아이디')

login


In [68]:
## 매무리

cur.close()
conn.close()